In [1]:
import time
import re
'''
Variables
description: 아뜰리에 요약정보
totalReviewPage: 전체 리뷰 페이지 수
reviews: 각 리뷰 데이터 -> reviewList: 리뷰 데이터 리스트
scores: 각 별점 데이터 -> scoreList: 별점 데이터 리스트
reviewPage: 가져올 리뷰 페이지 

Functions
getData: 페이지별 리뷰데이터
getKeywords: 각 아뜰리에별 테마키워드
'''

def getData(storeId,storeName, reviewPage) :
    scoreList = []
    reviewList = []    
    
    for i in range(0, reviewPage):
        searchURL = "https://store.naver.com/restaurants/detail?entry=pll&id="+storeId+"&query="+storeName+"&tab=receiptReview&tabPage="+str(i)
        driver.get(searchURL)
        
        reviews = driver.find_elements_by_class_name("review_txt")
        scores = driver.find_elements_by_class_name("score")
        
        for review in reviews:
            text = review.text
            only_BMP_pattern = re.compile("["u"\U00010000-\U0010FFFF""]+", flags=re.UNICODE)
            text = only_BMP_pattern.sub(r'', text)
            reviewList.append(text)
            
        for score in scores:
            scoreList.append(float(score.text)) #소수로 저장
            
    return reviewList,scoreList

def getKeywords(storeId,storeName):
    #keywordList = []
    strAll = ""
    description = ""
    
    searchURL = "https://store.naver.com/restaurants/detail?entry=pll&id="+storeId+"&query="+storeName+"&tab=main"
    driver.get(searchURL)
    keywords =  driver.find_elements_by_class_name("kwd")
    description = driver.find_elements_by_class_name("ellipsis_area")[0]
    description = (description.find_elements_by_tag_name("span")[0]).text
    
    for keyword in keywords:
        strAll += keyword.text
    
    return description,strAll


In [2]:
#아뜰리에 기본 데이터 불러오기 (아뜰리에명, 아뜰리에고유아이디)
from openpyxl import load_workbook

storeList = []
storeIdList = []

load_wb = load_workbook("fran_data.xlsx", data_only=True)
load_ws = load_wb['프렌차이즈 제거_전국 아뜰리에']

for i in range(2, 6227):
    storeList.append(load_ws.cell(row=i,column=2).value)
    storeIdList.append(load_ws.cell(row=i,column=3).value)

In [3]:
print(storeList[2002:3001])

['연지다방', '은실다방', '산장다방', '학다방', '은다방', '예다방', '화목다방', '더로드101', '평사리의아침', '쌍계명차', '금향다원', '우주총동원', '브릿지130', '카페포레', '카페하동', 'Planet1020', '흔적문화갤러리가있는커피숍', '그날', '쉼표하나', '하동스토리하우스카페', '윤슬당', '양보제과', '파이나무', '언더브릿지', '팥이야기', '달인꽈배기', '춤추는인도코끼리', '뱅상과자점', '카페노량', '도심다원', '브람스를좋아하세요', '알프스 밀밭', '화개락', '웰빙먹거리', '카페반달', '스마일카페', '카페오프닝', '카페이즈', '법향다원', '키친205', '함평성점숙꽃차', '돌카페', '로뎀나무카페', '녹유원', '카페인', '카페하우스', '살롱드바나나', '카페엘림', '거기카페', '신씨네베이커리', '카페프란', '양군팩토리', '더테이블', '서양과자점', '카페브릿지', '라임커피', '카페아마떼 함평점', '명희봉밀과자점', '영양재', '블랙빈', '카페두리샘', '잘만진빵', '해와달', '투팽', '스위치온', '커피로드333', '아마루', '어로우커피', '엘아르카 퀴에스', '161커피스튜디오', '가창옛날찐빵', '커피문화', '투썸플레이스대구화원점', '달성군청 사문진카페', '알벤토', '슬로우', '한빵울', '대새목장', '읍천리', '트라이톤 플랜트갤러리', '오르망', '나드리찐빵', '이흥용과자점 사직점', '텐퍼센트커피 부산시청본점', '보느파티쓰리', '어나더미네스', 'caffe gather', '구자윤과자점', '파리바게트거제현대', '아뜰리에마롱', '사이먼커피', '텐퍼센트커피 교대법원점', 'chic985', '디저트숲', '커피긱스', '디저트시네마', '오감베이커리', '하이드벗', '카페브루벡', '더인정', '파리바게뜨연산로터리점', '커피필라소피', '사랑愛(애)카페', '티포유', '빵85', '주택연회', '카페오

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import openpyxl


path = "resource/chromedriver"
driver = webdriver.Chrome()
write_wb = openpyxl.Workbook()
write_wb.create_sheet('아뜰리에_리뷰평점')
write_wb.create_sheet('아뜰리에_이름설명키워드')


write_ws = write_wb.get_sheet_by_name('아뜰리에_리뷰평점')
write_ws['A1'] = '아뜰리에명'
write_ws['B1'] = '리뷰'
write_ws['C1'] = '평점'

write_info = write_wb.get_sheet_by_name('아뜰리에_이름설명키워드')
write_info['A1'] = '아뜰리에명'
write_info['B1'] = '설명'
write_info['C1'] = '키워드'

cnt = 0 #리뷰개수가 30개~100개를 가지는 아뜰리에 넘버링

'''
TestData (description, count)
패션5 : O/440개 (리뷰가 100개 이상으므로 나오면 안됨)
르쁘띠푸: O/70개
까페베네치아: X/17개
달스윗:X/43개
'''

#테스팅 데이터
#storeIdList = [12851539,13156166,33144418,1372437122]
#storeList = ['패션5','르쁘띠푸','까페베네치아','달스윗'] 
keywordList = []
reviewList = []
scoreList = []
#크롤링 횟수 조절
countNum = 3001

#2000~2501
for i in range(2419,countNum):
    name = ""
    description = ""
    keywords = ""
    searchURL = "https://store.naver.com/restaurants/detail?entry=pll&id="+str(storeIdList[i])+"&query="+storeList[i]+"&tab=receiptReview"
    ss = 0
    try:
        while True:
            try:
                driver.get(searchURL)
                ss +=1
            except:
                driver.refresh()
            if ss == 1:
                break
            else:
                break
        ss = 0
        time.sleep(2)
        totalReviewPage = int(driver.find_elements_by_xpath("//*[@id='panel03']/div/div[2]/span")[0].text)            
        reviewPage = 0
                
        #리뷰 개수가 30개~100개까지 인 것만 
        if(totalReviewPage<4):
            continue
        elif(totalReviewPage>11):
            reviewPage = 3
        else: 
            reviewPage = totalReviewPage
        
        
    #리뷰데이터와 평점 데이터 
        while True: 
            try: 
                reviewList,scoreList = getData(str(storeIdList[i]),storeList[i],reviewPage)
                ss+=1
                #아뜰리에 설명 및 테마키워드
            except:
                driver.refresh()
            if ss==1:
                break
            else:
                break
        ss=0
        
        while True: 
            try: 
                description, keywords = getKeywords(str(storeIdList[i]),storeList[i])
                
                ss +=1
            except:
                driver.refresh()
            if ss==1:
                break
            else:
                break
        ss=0    
        name = storeList[i]
        cnt +=1
        print(cnt, name)
        
        #엑셀파일 셀에 저장        
        for k in range(0,len(reviewList)):
            review = reviewList[k]
            score = scoreList[k]
            write_len = write_ws.max_row
            write_ws.append([name, review,score])   
    except:
        driver.refresh()
    if len(name) > 1:
        write_info['A{}'.format(cnt+1)] = name
        write_info['B{}'.format(cnt+1)] = description
        write_info['C{}'.format(cnt+1)] = keywords
    write_wb.save('storeInfo2419to3001.xlsx')

<ipython-input-4-86f911d2b07f>:13: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  write_ws = write_wb.get_sheet_by_name('아뜰리에_리뷰평점')
<ipython-input-4-86f911d2b07f>:18: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  write_info = write_wb.get_sheet_by_name('아뜰리에_이름설명키워드')


1 더로드101
2 쌍계명차
3 우주총동원
4 카페포레
5 하동스토리하우스카페
6 키친205
7 커피로드333
8 161커피스튜디오
9 가창옛날찐빵
10 투썸플레이스대구화원점
11 읍천리
12 오르망
13 이흥용과자점 사직점
14 텐퍼센트커피 부산시청본점
15 보느파티쓰리
16 어나더미네스
17 caffe gather
18 구자윤과자점
19 파리바게트거제현대
20 아뜰리에마롱
21 사이먼커피
22 텐퍼센트커피 교대법원점
23 chic985
24 커피긱스
25 디저트시네마
26 오감베이커리
27 하이드벗
28 파리바게뜨연산로터리점
29 빵85
30 다온브레드
31 니드썸레스트
32 르봉뺑
33 리버레인
34 플로레
35 코미호미
36 나무아래오후
37 더스틸카페
38 꼬무네 카페
39 라틴정원
40 나무아래오후N
41 아트살롱가평
42 카페75
43 미스터박 베이커리카페
44 엠블럭
45 카페쁘라제르
46 스테이83
47 바이스테이션
48 라쿠나
49 제빵소덤 대성리점
50 홍종흔베이커리
51 배스킨라빈스산본중심가점
52 라붐커피 산본점
53 디마인커피로스터즈
54 숨맑은집
55 허니듀
56 손커피연구소 산본역점
57 부산케키
58 콩만한커피알만한케익
59 반월호수 온도
60 와플크림
61 신일룡의 호두파이
62 카페언타이
63 조셉파리
64 요요연연
65 이백
66 커피라끄
67 카페피노
68 이프테이
69 여울목카페
70 커피단월
71 게으른악어
72 듀레베이커리 남산점
73 듀레베이커리
74 세상상회
75 활옥동굴카페
76 타임
77 식스타임즈
78 샤인
79 배스킨라빈스충주국원점
80 모이까 금릉점
81 루암리
82 씨엘비베이커리
83 코롬방제과점
84 카페피어파이브
85 커피파나쉐 전망대점
86 카페닻
87 가비1935
88 쑥꿀레
89 커피창고로
90 배스킨라빈스목포상동점
91 그린하우스제과점
92 하루
93 미텐발트
94 코티지683
95 애뜨락
96 미스티
97 베니어 베이커리카페
98 빛담
99 늘솜당
100 릴리프커피
101 프로젝트 C
102

WebDriverException: Message: chrome not reachable
  (Session info: chrome=84.0.4147.125)
